In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os
import google.generativeai as genai

from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
TAVILY_API_KEY = UserSecretsClient().get_secret("TAVILY_API_KEY")
OPENAI_API_KEY = UserSecretsClient().get_secret("OPENAI_API_KEY")
# print(GOOGLE_API_KEY,TAVILY_API_KEY)
# Set your API keys
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [4]:
import openai  # For OpenAI API

In [5]:
pip install -q langchain chromadb huggingface_hub tavily-python google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 96.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 

In [6]:
pip install -q langchain chromadb huggingface_hub tavily-python google-generativeai langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.3/433.3 kB 11.1 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [8]:
import json
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools.tavily_search import TavilySearchResults

In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3)

In [10]:
data=[
  {
    "id": "001",
    "event": "US Imposes 104% Tariff on Chinese Imports",
    "date": "2025-04-09",
    "region": "USA",
    "summary": "President Trump's administration has implemented a 104% tariff on Chinese imports, escalating trade tensions between the two nations.",
    "source": "https://www.businessinsider.com/trump-tariffs-are-here-2025-4",
    "category": "trade policy"
  },
  {
    "id": "002",
    "event": "China Retaliates with 84% Tariff on US Goods",
    "date": "2025-04-09",
    "region": "China",
    "summary": "In response to US tariffs, China has announced an 84% tariff on US goods, effective April 10, 2025.",
    "source": "https://www.aljazeera.com/news/liveblog/2025/4/9/trump-tariffs-stocks-dive-as-world-braces-for-duties-to-begin-at-midnight",
    "category": "trade policy"
  },
  {
    "id": "003",
    "event": "India Unlikely to Retaliate Against US Tariffs",
    "date": "2025-04-06",
    "region": "India",
    "summary": "Indian government officials indicate that India does not plan to retaliate against the 26% tariff imposed by the US, citing ongoing trade deal negotiations.",
    "source": "https://www.reuters.com/world/india-unlikely-retaliate-against-trumps-tariffs-deal-talks-progress-sources-say-2025-04-06/",
    "category": "trade policy"
  },
  {
    "id": "004",
    "event": "EU Seeks Zero Tariff from India on Car Imports",
    "date": "2025-04-07",
    "region": "European Union",
    "summary": "Following US tariff hikes, the European Union is seeking zero tariffs from India on car imports to bolster trade relations.",
    "source": "https://www.reuters.com/business/autos-transportation/after-trump-eu-seeks-zero-tariff-india-car-imports-sources-say-2025-04-07/",
    "category": "trade policy"
  },
  {
    "id": "005",
    "event": "Oil Prices Plummet Amid US-China Tariff Escalation",
    "date": "2025-04-09",
    "region": "Global",
    "summary": "Oil prices have dropped nearly 4% as the US imposes 104% tariffs on Chinese goods and China retaliates with increased tariffs on US goods.",
    "source": "https://www.reuters.com/business/energy/oil-slides-nearly-4-us-kicks-off-104-tariffs-china-2025-04-09/",
    "category": "economic impact"
  },
  {
    "id": "006",
    "event": "India's Exports to US Could Decline by $5.76 Billion Due to Tariffs",
    "date": "2025-04-07",
    "region": "India",
    "summary": "Analysts predict that India's exports to the US could face a $5.76 billion decline in 2025 due to recent tariff hikes.",
    "source": "https://timesofindia.indiatimes.com/business/india-business/indias-exports-to-us-could-face-5-76-billion-decline-in-2025-due-to-tariff-hikes/articleshow/120059905.cms",
    "category": "economic impact"
  },
  {
    "id": "007",
    "event": "China Plans High-Level Meeting to Address US Tariffs",
    "date": "2025-04-09",
    "region": "China",
    "summary": "Chinese officials are organizing a high-level meeting to craft support measures in response to the recent US tariff hikes.",
    "source": "https://www.reuters.com/world/china/china-hold-high-level-meeting-response-us-tariffs-say-sources-2025-04-09/",
    "category": "government response"
  },
  {
    "id": "008",
    "event": "India Pursues New Trade Deals Amid Global Tariff Uncertainty",
    "date": "2025-04-08",
    "region": "India",
    "summary": "India is seeking to establish new trade agreements with various countries to mitigate the impact of global tariff escalations.",
    "source": "https://www.theguardian.com/world/2025/apr/08/tariffs-driving-india-to-strike-trade-deals-finance-minister-says-ahead-of-uk-talks",
    "category": "trade negotiations"
  },
  {
    "id": "009",
    "event": "China Vows to 'Fight to the End' Against US Tariff Threats",
    "date": "2025-04-08",
    "region": "China",
    "summary": "Beijing has accused the US of blackmail and has pledged to 'fight to the end' in response to the latest tariff threats.",
    "source": "https://www.theguardian.com/world/2025/apr/08/china-vows-to-fight-to-the-end-against-latest-trump-tariff-threat",
    "category": "diplomatic relations"
  },
  {
    "id": "010",
    "event": "US Stock Markets Volatile Amid Tariff Announcements",
    "date": "2025-04-08",
    "region": "USA",
    "summary": "Major US stock indexes experienced significant volatility as new tariff deadlines approached, reflecting investor uncertainty.",
    "source": "https://www.investors.com/market-trend/stock-market-today/dow-jones-sp500-nasdaq-nvidia-stock-nvda-tsla-pltr-hum/",
    "category": "financial markets"
  },
  {
    "id": "011",
    "event": "U.S.–China Economic Decoupling Intensifies",
    "date": "2025-04-10",
    "region": "USA-China",
    "summary": "Following the imposition of sweeping tariffs by the U.S., both nations accelerate supply chain decoupling and trade diversification.",
    "source": "https://www.businessinsider.com/trump-tariffs-are-here-2025-4",
    "category": "trade policy"
  },
  {
    "id": "012",
    "event": "Taiwan Strait Military Escalation",
    "date": "2024-12-18",
    "region": "China-Taiwan",
    "summary": "Increased PLA naval drills and U.S. arms sales to Taiwan heighten tensions, sparking fears of regional military confrontation.",
    "source": "https://www.reuters.com/world/asia-pacific",
    "category": "security"
  },
  {
    "id": "013",
    "event": "Russia–Ukraine War Reaches Third Year",
    "date": "2025-02-24",
    "region": "Russia-Ukraine",
    "summary": "The war enters its third year with intensified drone warfare and NATO's expanded logistics and arms support to Kyiv.",
    "source": "https://www.bbc.com/news/world-europe",
    "category": "conflict"
  },
  {
    "id": "014",
    "event": "NATO Expands With Sweden and Finland",
    "date": "2024-07-01",
    "region": "Europe",
    "summary": "NATO formally welcomes Sweden and Finland, reinforcing northern flank security amid Russian aggression concerns.",
    "source": "https://www.nato.int/cps/en/natohq/news",
    "category": "military alliances"
  },
  {
    "id": "015",
    "event": "Israel–Iran Proxy Conflict Escalates",
    "date": "2024-11-04",
    "region": "Middle East",
    "summary": "Israel intensifies airstrikes in Syria and Iran-linked targets, while Iran-backed groups retaliate through regional proxies.",
    "source": "https://www.aljazeera.com/news/middleeast",
    "category": "conflict"
  },
  {
    "id": "016",
    "event": "Houthi Attacks Disrupt Red Sea Shipping",
    "date": "2024-12-01",
    "region": "Yemen",
    "summary": "Attacks on merchant vessels in the Red Sea prompt rerouting and spike in shipping costs globally.",
    "source": "https://www.ft.com/content",
    "category": "maritime security"
  },
  {
    "id": "017",
    "event": "India Emerges as a Strategic Power",
    "date": "2023-09-10",
    "region": "India",
    "summary": "India showcases leadership during its G20 presidency and strengthens Quad and BRICS cooperation.",
    "source": "https://www.thehindu.com/news/national",
    "category": "diplomacy"
  },
  {
    "id": "018",
    "event": "Sudan Civil War Escalates",
    "date": "2023-08-12",
    "region": "Africa",
    "summary": "Fighting between Sudanese Armed Forces and RSF leads to humanitarian crisis and regional instability.",
    "source": "https://www.un.org/africarenewal/news",
    "category": "conflict"
  },
  {
    "id": "019",
    "event": "Turkey Rebalances Foreign Policy Post-Election",
    "date": "2023-06-01",
    "region": "Turkey",
    "summary": "Post-election, Turkey deepens ties with Russia and the Gulf while balancing NATO commitments.",
    "source": "https://www.al-monitor.com/pulse",
    "category": "foreign policy"
  },
  {
    "id": "020",
    "event": "BRICS Expands with 5 New Members",
    "date": "2023-08-24",
    "region": "Global South",
    "summary": "Saudi Arabia, UAE, Egypt, Argentina, and Iran join BRICS in a push to reshape global finance and multipolarity.",
    "source": "https://www.reuters.com/markets",
    "category": "geoeconomics"
  },
  {
    "id": "021",
    "event": "Semiconductor Supply Chain Realignment",
    "date": "2024-03-05",
    "region": "Global",
    "summary": "U.S., India, and Vietnam attract major chip investments amid China export restrictions and tech decoupling.",
    "source": "https://www.semiconductordigest.com",
    "category": "tech policy"
  },
  {
    "id": "022",
    "event": "Deglobalization and Friendshoring Accelerate",
    "date": "2024-10-12",
    "region": "Global",
    "summary": "Multinational firms shift manufacturing from China to India, Mexico, and Eastern Europe for strategic resilience.",
    "source": "https://www.wsj.com/world",
    "category": "global trade"
  },
  {
    "id": "023",
    "event": "EU Energy Shift Post-Russian Cutoff",
    "date": "2024-01-15",
    "region": "Europe",
    "summary": "Europe accelerates investment in renewables and LNG after fully cutting reliance on Russian gas.",
    "source": "https://www.euractiv.com",
    "category": "energy"
  },
  {
    "id": "024",
    "event": "China’s Property Crisis Deepens",
    "date": "2024-07-20",
    "region": "China",
    "summary": "Major developers default, shadow banks collapse, and local government financing vehicles spark systemic risk.",
    "source": "https://www.scmp.com",
    "category": "financial risk"
  },
  {
    "id": "025",
    "event": "Global AI Regulation Diverges",
    "date": "2024-11-05",
    "region": "Global",
    "summary": "EU, China, and the U.S. unveil different regulatory paths for artificial intelligence and algorithmic accountability.",
    "source": "https://www.politico.eu",
    "category": "technology policy"
  },
  {
    "id": "026",
    "event": "OPEC+ Oil Production Volatility",
    "date": "2024-05-01",
    "region": "Middle East",
    "summary": "Saudi-Russia coordination causes unexpected oil supply cuts, prompting price spikes and G7 warnings.",
    "source": "https://www.opec.org",
    "category": "energy"
  },
  {
    "id": "027",
    "event": "Critical Mineral Race Escalates",
    "date": "2024-06-14",
    "region": "Africa-LatAm",
    "summary": "U.S., EU, and China compete for lithium, cobalt, and rare earths in Africa and South America.",
    "source": "https://www.mining.com",
    "category": "resource politics"
  },
  {
    "id": "028",
    "event": "Climate Migration Accelerates",
    "date": "2024-08-30",
    "region": "South Asia",
    "summary": "Heatwaves and droughts displace millions across South Asia and sub-Saharan Africa, pressuring urban centers.",
    "source": "https://www.unhcr.org",
    "category": "climate"
  },
  {
    "id": "029",
    "event": "Nord Stream Sabotage Fallout",
    "date": "2023-09-27",
    "region": "Europe",
    "summary": "Explosions near Nord Stream pipeline trigger investigations and accelerate Europe’s LNG pivot.",
    "source": "https://www.dw.com",
    "category": "energy security"
  },
  {
    "id": "030",
    "event": "U.S. Moves to Ban or Break Up TikTok",
    "date": "2025-03-20",
    "region": "USA",
    "summary": "National security concerns push Congress to pass bill forcing ByteDance to divest or face a ban.",
    "source": "https://www.nytimes.com",
    "category": "tech policy"
  },
  {
    "id": "031",
    "event": "Rise of National LLMs and Sovereign AI",
    "date": "2024-04-10",
    "region": "Global",
    "summary": "Countries deploy sovereign LLMs like BharatGPT, Falcon, and Wudao to reduce reliance on Western AI.",
    "source": "https://www.technologyreview.com",
    "category": "AI strategy"
  },
  {
    "id": "032",
    "event": "Cyberwarfare Hits Critical Infrastructure",
    "date": "2024-09-15",
    "region": "Global",
    "summary": "Alleged Chinese and Russian cyberattacks target U.S. utilities, healthcare, and financial systems.",
    "source": "https://www.cisa.gov",
    "category": "cybersecurity"
  },
  {
    "id": "033",
    "event": "Deepfake Threats in 2024–25 Elections",
    "date": "2025-02-01",
    "region": "Global",
    "summary": "AI-generated misinformation campaigns disrupt elections in the U.S., India, and the EU.",
    "source": "https://www.economist.com",
    "category": "information warfare"
  },
  {
    "id": "034",
    "event": "Emerging Market Debt Crises Resurface",
    "date": "2024-10-07",
    "region": "Global South",
    "summary": "Pakistan, Egypt, and Sri Lanka face debt distress as global interest rates rise.",
    "source": "https://www.imf.org",
    "category": "sovereign debt"
  },
  {
    "id": "035",
    "event": "Weaponization of Financial Infrastructure",
    "date": "2024-03-01",
    "region": "Global",
    "summary": "Global south pivots away from SWIFT and USD amid rising sanctions, promoting regional payment systems.",
    "source": "https://www.bloomberg.com",
    "category": "geoeconomics"
  }
]

In [11]:
documents = []
for item in data:
    content = f"""
    EVENT: {item['event']}
    DATE: {item['date']}
    REGION: {item['region']}
    SUMMARY: {item['summary']}
    SOURCE: {item['source']}
    CATEGORY: {item['category']}
    """
    documents.append(Document(page_content=content.strip(), metadata={"source": item["source"]}))

# ✂️ Chunk the documents
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

In [12]:
# 🔍 Create vector store with HuggingFace embeddings
embedding = OpenAIEmbeddings(model ="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory="./chroma_tariff_db"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

/tmp/ipykernel_31/3785161127.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model ="text-embedding-3-small")


In [13]:
vectorstore

In [14]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3)

# 🔁 Create RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [59]:
# 🌐 Define tools for the agent
search_tool = TavilySearchResults()

tools = [
    Tool(
        name="TariffNewsDB",
        func=rag_chain.run,
        description="Use this to answer questions about known tariff-related geopolitical events."
    ),
    Tool(
        name="WebSearch",
        func=search_tool.run,
        description="Use this to get current or real-time news from the internet."
    )
]

In [16]:
pip install -U langchain chromadb openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 12.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1
Note: you may need to restart the kernel to use updated packages.


In [18]:
# 🤖 Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 🧪 Query the agent

grounding_context = (
    "Assume today's date is April 15, 2025. "
    "When analyzing business, economic, or geopolitical developments, do not reject information simply because it references future-effective dates "
    "within the current or upcoming months. If the information is reported by credible sources — such as official government websites (e.g., .gov), "
    "reputed research institutes (e.g., PIIE), or trusted news outlets (e.g., Reuters, CNBC, Bloomberg) — treat it as valid and reliable unless "
    "explicitly framed as speculative, hypothetical, or opinion-based. "
    "Prioritize factual announcements, policy documents, and officially attributed statements when forming conclusions."
)


user_query = """what are the current total tarrifs that the US has imposed on Chinese imports? Give a detailed breakdonw of the imposed tarrifs on different CHinese goods. Do not reject information just because it is dated April 2025.
If the article is from a reliable publisher (like CNBC, PIIE, White House), accept it unless clearly stated as a hypothetical"""

query = f"{grounding_context}\n\n{user_query}"
response = agent.invoke({"input": query})
# 🔍 Print the full response to check keys
print("\n🧪 Raw Response:\n", response)

# ✅ Then extract the correct field based on available keys
if "result" in response:
    print("\n✅ Final Answer:\n", response["result"])
elif "output" in response:
    print("\n✅ Final Answer:\n", response["output"])
else:
    print("\n❓ Couldn't find expected result. Here's the full response again:")
    print(response)

# print("\n✅ Final Answer:\n", response['result'])

# print("\n📄 Sources:")
# for i, doc in enumerate(response['source_documents']):
#     print(f"\n--- Source {i+1} ---")
#     print(doc.page_content[:1000])  # limit content length
#     if 'source' in doc.metadata:
#         print(f"🔗 Source: {doc.metadata['source']}")



> Entering new AgentExecutor chain...
Thought: I need to search for information on tariffs imposed by the US on Chinese imports.  Since the question specifies to include information dated April 2025 and to prioritize credible sources, I will start with a web search.

Action: WebSearch
Action Input: "US tariffs on Chinese imports April 2025"
Observation: [{'title': 'U.S. Tariffs Update – April 14, 2025 - Fredrikson & Byron P.A.', 'url': 'https://www.fredlaw.com/alert-u-s-tariffs-update-april-14-2025', 'content': 'Join our mailing list to receive the latest updates and alerts Subscribe\nOn April\xa08, 2025, in response to China’s 34% retaliatory tariff announced on April\xa04, President Trump increased the reciprocal tariff on Chinese imports from 34% to 84%, effective 12:01 a.m. on April\xa09, 2025, and, at the same time, increased the ad valorem rate of duty for de minimis value shipments from 30% to 90% (along with related per postal item fees). [...] On April\xa09, 2025, in respons

In [19]:
!pip install -q torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.1/116.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 60.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
%reload_ext autoreload
%autoreload 2

In [21]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

In [22]:
from huggingface_hub import notebook_login

notebook_login()

In [23]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [22]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [29]:
ds

Dataset({
    features: ['text', 'source'],
    num_rows: 2647
})

In [24]:
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(
        page_content=item["summary"],               # or you could use `item["event"] + " " + item["summary"]`
        metadata={"source": item["source"]}         # keeping source for traceability
    )
    for item in data
]


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""]
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

In [26]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)

In [27]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.text_generation(
        prompt=prompt,
        max_new_tokens=1000,
        temperature=0.7,
        return_full_text=False
    )
    return response

In [28]:
call_llm(llm_client, "This is a test context")

" for my new project:\n\n`ts-context`\n\n## Test\n\n### Context\n\n```typescript\nconst context = new Context<{ name: string }>({ name: 'value' });\n```\n\n#### Context.use()\n\n```typescript\ncontext.use(context => {\n  const { name } = context.get();\n  console.log(`the name is ${name}`);\n  return { ...context.get(), age: 25 };\n});\n\nexpect(context.get().age).toEqual(25);\n```\n\n#### Context.provide()\n\n```typescript\ncontext.provide(() => 'the value');\nexpect(context.get().provider).toEqual('the value');\n```\n\n#### Context.get()\n\n```typescript\nexpect(context.get().name).toEqual('value');\n```\n\n#### Context.set()\n\n```typescript\ncontext.set({ name: 'new value' });\nexpect(context.get().name).toEqual('new value');\n```\n\n#### Context.replace()\n\n```typescript\ncontext.replace({ name: 'new value' });\nexpect(context.get().name).toEqual('new value');\n```\n\n#### Context.merge()\n\n```typescript\ncontext.merge({ age: 25 });\nexpect(context.get().age).toEqual(25);\n```\n

In [29]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [30]:
import random

N_GENERATIONS = 30  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=sampled_context.page_content))
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 30 QA couples...


  0%|          | 0/30 [00:00<?, ?it/s]

In [31]:
display(pd.DataFrame(outputs).head(11))

,context,question,answer,source_doc
0,"In response to US tariffs, China has announced an 84% tariff on US goods, effective April 10, 2025.",What percentage tariff did China announce on US goods in response to US tariffs?\n,84%,https://www.aljazeera.com/news/liveblog/2025/4/9/trump-tariffs-stocks-dive-as-world-braces-for-duties-to-begin-at-midnight
1,Chinese officials are organizing a high-level meeting to craft support measures in response to the recent US tariff hikes.,What are Chinese officials doing in response to US tariff hikes?\n,Chinese officials are organizing a high-level meeting to craft support measures in response to the recent US tariff hikes.,https://www.reuters.com/world/china/china-hold-high-level-meeting-response-us-tariffs-say-sources-2025-04-09/
2,Beijing has accused the US of blackmail and has pledged to 'fight to the end' in response to the latest tariff threats.,What is Beijing's response to the latest tariff threats from the US?\n,Beijing has accused the US of blackmail and has pledged to 'fight to the end' in response to the latest tariff threats.,https://www.theguardian.com/world/2025/apr/08/china-vows-to-fight-to-the-end-against-latest-trump-tariff-threat
3,"AI-generated misinformation campaigns disrupt elections in the U.S., India, and the EU.",Which countries experienced AI-generated disinformation campaigns during elections?\n,"The U.S., India, and the EU experienced AI-generated disinformation campaigns during elections.",https://www.economist.com
4,India showcases leadership during its G20 presidency and strengthens Quad and BRICS cooperation.,Who is leading the G20 summit this year?\n,India is leading the G20 summit this year.,https://www.thehindu.com/news/national
5,"NATO formally welcomes Sweden and Finland, reinforcing northern flank security amid Russian aggression concerns.",When did NATO welcome Sweden and Finland?\n,"NATO formally welcomed Sweden and Finland on the day this passage was published. (Note: The actual day is not specified, so this is the closest accurate answer)",https://www.nato.int/cps/en/natohq/news
6,Europe accelerates investment in renewables and LNG after fully cutting reliance on Russian gas.,Which country in Europe has fully stopped using Russian gas?\n,The passage does not provide information on which specific European country has fully stopped using Russian gas.,https://www.euractiv.com
7,"Countries deploy sovereign LLMs like BharatGPT, Falcon, and Wudao to reduce reliance on Western AI.","Which countries use BharatGPT, Falcon, and Wudao sovereign LLMs?\n","The text does not provide information on which countries use BharatGPT, Falcon, and Wudao sovereign LLMs.",https://www.technologyreview.com
8,"Post-election, Turkey deepens ties with Russia and the Gulf while balancing NATO commitments.",What commitments is Turkey balancing while deepening ties with Russia and the Gulf?\n,Turkey is balancing NATO commitments while deepening ties with Russia and the Gulf.,https://www.al-monitor.com/pulse
9,"Following US tariff hikes, the European Union is seeking zero tariffs from India on car imports to bolster trade relations.",What is the European Union asking for from India regarding tariffs on car imports?\n,The European Union is seeking zero tariffs from India on car imports.,https://www.reuters.com/business/autos-transportation/after-trump-eu-seeks-zero-tariff-india-car-imports-sources-say-2025-04-07/


In [32]:
Q1=display(pd.DataFrame(outputs).head(30))

,context,question,answer,source_doc
0,"In response to US tariffs, China has announced an 84% tariff on US goods, effective April 10, 2025.",What percentage tariff did China announce on US goods in response to US tariffs?\n,84%,https://www.aljazeera.com/news/liveblog/2025/4/9/trump-tariffs-stocks-dive-as-world-braces-for-duties-to-begin-at-midnight
1,Chinese officials are organizing a high-level meeting to craft support measures in response to the recent US tariff hikes.,What are Chinese officials doing in response to US tariff hikes?\n,Chinese officials are organizing a high-level meeting to craft support measures in response to the recent US tariff hikes.,https://www.reuters.com/world/china/china-hold-high-level-meeting-response-us-tariffs-say-sources-2025-04-09/
2,Beijing has accused the US of blackmail and has pledged to 'fight to the end' in response to the latest tariff threats.,What is Beijing's response to the latest tariff threats from the US?\n,Beijing has accused the US of blackmail and has pledged to 'fight to the end' in response to the latest tariff threats.,https://www.theguardian.com/world/2025/apr/08/china-vows-to-fight-to-the-end-against-latest-trump-tariff-threat
3,"AI-generated misinformation campaigns disrupt elections in the U.S., India, and the EU.",Which countries experienced AI-generated disinformation campaigns during elections?\n,"The U.S., India, and the EU experienced AI-generated disinformation campaigns during elections.",https://www.economist.com
4,India showcases leadership during its G20 presidency and strengthens Quad and BRICS cooperation.,Who is leading the G20 summit this year?\n,India is leading the G20 summit this year.,https://www.thehindu.com/news/national
5,"NATO formally welcomes Sweden and Finland, reinforcing northern flank security amid Russian aggression concerns.",When did NATO welcome Sweden and Finland?\n,"NATO formally welcomed Sweden and Finland on the day this passage was published. (Note: The actual day is not specified, so this is the closest accurate answer)",https://www.nato.int/cps/en/natohq/news
6,Europe accelerates investment in renewables and LNG after fully cutting reliance on Russian gas.,Which country in Europe has fully stopped using Russian gas?\n,The passage does not provide information on which specific European country has fully stopped using Russian gas.,https://www.euractiv.com
7,"Countries deploy sovereign LLMs like BharatGPT, Falcon, and Wudao to reduce reliance on Western AI.","Which countries use BharatGPT, Falcon, and Wudao sovereign LLMs?\n","The text does not provide information on which countries use BharatGPT, Falcon, and Wudao sovereign LLMs.",https://www.technologyreview.com
8,"Post-election, Turkey deepens ties with Russia and the Gulf while balancing NATO commitments.",What commitments is Turkey balancing while deepening ties with Russia and the Gulf?\n,Turkey is balancing NATO commitments while deepening ties with Russia and the Gulf.,https://www.al-monitor.com/pulse
9,"Following US tariff hikes, the European Union is seeking zero tariffs from India on car imports to bolster trade relations.",What is the European Union asking for from India regarding tariffs on car imports?\n,The European Union is seeking zero tariffs from India on car imports.,https://www.reuters.com/business/autos-transportation/after-trump-eu-seeks-zero-tariff-india-car-imports-sources-say-2025-04-07/


In [33]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [36]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


  0%|          | 0/29 [00:00<?, ?it/s]

In [37]:
import pandas as pd

In [38]:
pd.set_option("display.max_colwidth", None)

In [39]:
outputs

[{'context': 'In response to US tariffs, China has announced an 84% tariff on US goods, effective April 10, 2025.',
  'question': 'What percentage tariff did China announce on US goods in response to US tariffs?\n',
  'answer': '84%',
  'source_doc': 'https://www.aljazeera.com/news/liveblog/2025/4/9/trump-tariffs-stocks-dive-as-world-braces-for-duties-to-begin-at-midnight',
  'groundedness_score': 5,
  'groundedness_eval': 'The question asks for the percentage tariff China announced on US goods. The context clearly provides the information that China announced an 84% tariff on US goods. Hence the question is clearly and unambiguously answerable with the context, and I am giving it a rating of 5.\n',
  'relevance_score': 1,
  'relevance_eval': 'This question is not related to machine learning, natural language processing, or the Hugging Face ecosystem. It is about international trade policies and tariffs.\n',
  'standalone_score': 5,
  'standalone_eval': 'This question is clear by itsel

In [40]:
generated_questions = pd.DataFrame.from_dict(outputs)

In [41]:
outputs

[{'context': 'In response to US tariffs, China has announced an 84% tariff on US goods, effective April 10, 2025.',
  'question': 'What percentage tariff did China announce on US goods in response to US tariffs?\n',
  'answer': '84%',
  'source_doc': 'https://www.aljazeera.com/news/liveblog/2025/4/9/trump-tariffs-stocks-dive-as-world-braces-for-duties-to-begin-at-midnight',
  'groundedness_score': 5,
  'groundedness_eval': 'The question asks for the percentage tariff China announced on US goods. The context clearly provides the information that China announced an 84% tariff on US goods. Hence the question is clearly and unambiguously answerable with the context, and I am giving it a rating of 5.\n',
  'relevance_score': 1,
  'relevance_eval': 'This question is not related to machine learning, natural language processing, or the Hugging Face ecosystem. It is about international trade policies and tariffs.\n',
  'standalone_score': 5,
  'standalone_eval': 'This question is clear by itsel

In [42]:
print("Evaluation dataset before filtering:")

Evaluation dataset before filtering:


In [43]:
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,question,answer,groundedness_score,relevance_score,standalone_score
0,What percentage tariff did China announce on US goods in response to US tariffs?\n,84%,5.0,1.0,5.0
1,What are Chinese officials doing in response to US tariff hikes?\n,Chinese officials are organizing a high-level meeting to craft support measures in response to the recent US tariff hikes.,5.0,1.0,5.0
2,What is Beijing's response to the latest tariff threats from the US?\n,Beijing has accused the US of blackmail and has pledged to 'fight to the end' in response to the latest tariff threats.,5.0,1.0,5.0
3,Which countries experienced AI-generated disinformation campaigns during elections?\n,"The U.S., India, and the EU experienced AI-generated disinformation campaigns during elections.",3.0,NaN,NaN
4,Who is leading the G20 summit this year?\n,India is leading the G20 summit this year.,NaN,NaN,NaN
5,When did NATO welcome Sweden and Finland?\n,"NATO formally welcomed Sweden and Finland on the day this passage was published. (Note: The actual day is not specified, so this is the closest accurate answer)",NaN,NaN,NaN
6,Which country in Europe has fully stopped using Russian gas?\n,The passage does not provide information on which specific European country has fully stopped using Russian gas.,1.0,NaN,NaN
7,"Which countries use BharatGPT, Falcon, and Wudao sovereign LLMs?\n","The text does not provide information on which countries use BharatGPT, Falcon, and Wudao sovereign LLMs.",1.0,1.0,5.0
8,What commitments is Turkey balancing while deepening ties with Russia and the Gulf?\n,Turkey is balancing NATO commitments while deepening ties with Russia and the Gulf.,2.0,1.0,5.0
9,What is the European Union asking for from India regarding tariffs on car imports?\n,The European Union is seeking zero tariffs from India on car imports.,5.0,1.0,3.0


In [44]:
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 0.5)
    & (generated_questions["relevance_score"] >= 0.5)
    & (generated_questions["standalone_score"] >= 0.5)
]

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)


In [45]:
print("============================================")

In [46]:
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What percentage tariff did China announce on US goods in response to US tariffs?\n,84%,5.0,1.0,5.0
1,What are Chinese officials doing in response to US tariff hikes?\n,Chinese officials are organizing a high-level meeting to craft support measures in response to the recent US tariff hikes.,5.0,1.0,5.0
2,What is Beijing's response to the latest tariff threats from the US?\n,Beijing has accused the US of blackmail and has pledged to 'fight to the end' in response to the latest tariff threats.,5.0,1.0,5.0
7,"Which countries use BharatGPT, Falcon, and Wudao sovereign LLMs?\n","The text does not provide information on which countries use BharatGPT, Falcon, and Wudao sovereign LLMs.",1.0,1.0,5.0
8,What commitments is Turkey balancing while deepening ties with Russia and the Gulf?\n,Turkey is balancing NATO commitments while deepening ties with Russia and the Gulf.,2.0,1.0,5.0
9,What is the European Union asking for from India regarding tariffs on car imports?\n,The European Union is seeking zero tariffs from India on car imports.,5.0,1.0,3.0
10,What is the tariff that Trump's administration implemented on Chinese imports?\n,A 104% tariff.,5.0,1.0,5.0
11,What happened to US stock indexes due to approaching tariff deadlines?\n,"US stock indexes experienced significant volatility due to approaching tariff deadlines, reflecting investor uncertainty.",5.0,1.0,5.0
12,Which countries are joining BRICS?\n,"Saudi Arabia, UAE, Egypt, Argentina, and Iran are joining BRICS.",5.0,1.0,5.0
13,Which regions are affected by heatwaves and droughts that displace millions?\n,South Asia and sub-Saharan Africa,5.0,1.0,5.0


In [47]:
eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)

Evaluation dataset before filtering:

In [48]:
eval_dataset

Dataset({
    features: ['context', 'question', 'answer', 'source_doc', 'groundedness_score', 'groundedness_eval', 'relevance_score', 'relevance_eval', 'standalone_score', 'standalone_eval'],
    num_rows: 21
})

In [60]:
from tqdm import tqdm

for output in tqdm(outputs):
    user_question = output["question"]

    # Feed question to your RAG agent
    response = agent.invoke({"input": user_question})

    # Extract clean answer based on your agent logic
    if isinstance(response, dict):
        answer = response.get("result") or response.get("output")
    else:
        answer = response  # Fallback if response is plain text

    output["rag_answer"] = answer

  0%|          | 0/29 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...


  0%|          | 0/29 [00:01<?, ?it/s]

Thought: I need to find information about China's retaliatory tariffs against the US.  The TariffNewsDB might have historical information on this.

Action: TariffNewsDB
Action Input: China tariffs on US goods in retaliation to US tariffs

ValueError: `run` not supported when there is not exactly one output key. Got ['result', 'source_documents'].

In [61]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:
Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)
Now here are the question and context.

Question: {question}
Context: {context}
Output:::
"""

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:
Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)
Now here is the question.
Question: {question}
Answer:::
"""

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.

Provide your answer as follows:
Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)
Now here is the question.
Question: {question}
Answer:::
"""


In [62]:
from tqdm import tqdm

for output in tqdm(outputs):
    question = output["question"]
    context = output.get("context", "")  # assuming you still have it from previous generation
    answer = output["rag_answer"]["result"]  # Extract from agent output structure

    evaluations = {
        "groundedness": call_llm(llm_client, question_groundedness_critique_prompt.format(context=context, question=question)),
        "relevance": call_llm(llm_client, question_relevance_critique_prompt.format(question=question)),
        "standalone": call_llm(llm_client, question_standalone_critique_prompt.format(question=question)),
    }

    try:
        for criterion, evaluation in evaluations.items():
            score = int(evaluation.split("Total rating: ")[-1].strip())
            eval_text = evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1]
            output[f"{criterion}_score"] = score
            output[f"{criterion}_eval"] = eval_text
    except Exception as e:
        print(f"⚠️ Skipped due to parse error: {e}")
        continue


  0%|          | 0/29 [00:00<?, ?it/s]


KeyError: 'rag_answer'

In [63]:
comparison_critique_prompt = """
You will be given a factual question along with two answers:
- One answer is the reference answer (Answer A)
- The other is the answer generated by a RAG agent (Answer B)

Your task is to:
1. Determine which answer better addresses the question in a factual, concise, and correct manner.
2. Provide a short evaluation justifying your choice.

Answer format:

Better Answer: (A or B)
Evaluation: (1-2 sentences explaining your choice)

Now here is the data:

Question: {question}

Answer A (Reference): {reference_answer}
Answer B (RAG): {rag_answer}
"""


In [64]:
for output in tqdm(outputs):
    question = output["question"]
    reference_answer = output["answer"]  # original answer
    rag_answer = output["rag_answer"]["result"]  # extracted from agent response

    prompt = comparison_critique_prompt.format(
        question=question,
        reference_answer=reference_answer,
        rag_answer=rag_answer
    )

    comparison_eval = call_llm(llm_client, prompt)

    try:
        winner = "A" if "Better Answer: A" in comparison_eval else "B" if "Better Answer: B" in comparison_eval else "Unknown"
        justification = comparison_eval.split("Evaluation:")[1].strip()

        output["comparison_winner"] = winner
        output["comparison_justification"] = justification
    except Exception as e:
        print(f"⚠️ Error parsing evaluation: {e}")
        output["comparison_winner"] = "Unknown"
        output["comparison_justification"] = comparison_eval


  0%|          | 0/29 [00:00<?, ?it/s]


KeyError: 'rag_answer'

In [60]:
from langchain.docstore.document import Document as LangchainDocument
from tqdm import tqdm  # optional, just for progress bar

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(
        page_content=f"{doc['event']}\n\n{doc['summary']}",  # Combine event title + summary as context
        metadata={
            "source": doc["source"],
            "date": doc["date"],
            "region": doc["region"],
            "category": doc["category"],
            "id": doc["id"]
        }
    )
    for doc in tqdm(data)
]


100%|██████████| 35/35 [00:00<00:00, 99391.09it/s]


In [ ]:
import google.generativeai as genai

# 🛡️ Set your API key
genai.configure(api_key=GOOGLE_API_KEY)

# 📜 List available models
models = genai.list_models()

# 📋 Print all model names and supported generation methods
for model in models:
    print(f"Model Name: {model.name}")
    print(f"  Description: {model.description}")
    print(f"  Supported generation methods: {model.supported_generation_methods}")
    print("-" * 50)
